<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: старший преподаватель Факультета Компьютерных Наук НИУ ВШЭ Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее  задание № 9</center>
## <center>Vowpal Wabbit в задаче идентификации пользователей по посещенным веб-сайтам</center>

[Веб-форма](https://goo.gl/forms/rnUMG9jXTi2qJ6mk2) для ответов.

In [30]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

**Считайте данные [соревнования](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/data) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [31]:
train_df = pd.read_csv('../../data/train_sessions.csv')
test_df = pd.read_csv('../../data/test_sessions.csv')

In [32]:
train_df

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
0,1,8,2014-01-04 08:44:50,11.0,2014-01-04 08:44:50,82.0,2014-01-04 08:45:19,68.0,2014-01-04 08:45:25,8.0,...,2014-01-04 08:45:51,8403.0,2014-01-04 08:45:51,932.0,2014-01-04 08:45:53,3260.0,2014-01-04 08:45:53,8.0,2014-01-04 08:45:53,1845
1,2,111,2014-03-18 10:33:20,78.0,2014-03-18 10:33:31,151.0,2014-03-18 10:33:31,111.0,2014-03-18 10:33:31,1401.0,...,2014-03-18 10:33:32,1375.0,2014-03-18 10:33:32,38.0,2014-03-18 10:33:32,1401.0,2014-03-18 10:33:32,97.0,2014-03-18 10:33:34,3322
2,3,11,2014-12-02 13:13:41,3187.0,2014-12-02 13:13:41,132.0,2014-12-02 13:13:42,496.0,2014-12-02 13:13:42,1969.0,...,2014-12-02 13:13:45,3187.0,2014-12-02 13:13:45,82.0,2014-12-02 13:13:46,3191.0,2014-12-02 13:13:46,3184.0,2014-12-02 13:13:47,2003
3,4,668,2014-02-14 15:16:45,1965.0,2014-02-14 15:17:13,598.0,2014-02-14 15:20:47,1965.0,2014-02-14 15:21:13,284.0,...,2014-02-14 15:21:14,38.0,2014-02-14 15:21:14,4451.0,2014-02-14 15:21:14,4537.0,2014-02-14 15:21:15,11.0,2014-02-14 15:21:15,1373
4,5,1943,2014-03-17 15:19:40,1943.0,2014-03-17 15:20:10,1943.0,2014-03-17 15:21:40,1943.0,2014-03-17 15:22:10,1943.0,...,2014-03-17 15:22:39,1952.0,2014-03-17 15:22:41,1943.0,2014-03-17 15:22:41,1943.0,2014-03-17 15:22:42,1943.0,2014-03-17 15:22:43,1737
5,6,2941,2013-02-12 12:09:43,2951.0,2013-02-12 12:09:44,2958.0,2013-02-12 12:09:46,2993.0,2013-02-12 12:09:54,162.0,...,2013-02-12 12:09:55,85.0,2013-02-12 12:09:55,7.0,2013-02-12 12:09:56,2967.0,2013-02-12 12:09:56,2967.0,2013-02-12 12:09:58,1173
6,7,9996,2014-03-19 10:15:50,9996.0,2014-03-19 10:16:50,307.0,2014-03-19 10:17:19,9996.0,2014-03-19 10:17:20,280.0,...,2014-03-19 10:17:50,9996.0,2014-03-19 10:18:20,307.0,2014-03-19 10:18:41,9996.0,2014-03-19 10:18:50,9996.0,2014-03-19 10:19:20,440
7,8,19823,2013-04-12 10:52:36,19823.0,2013-04-12 10:52:37,1510.0,2013-04-12 10:53:37,32.0,2013-04-12 10:54:49,19808.0,...,2013-04-12 10:55:08,567.0,2013-04-12 10:55:57,567.0,2013-04-12 10:55:58,654.0,2013-04-12 10:55:59,567.0,2013-04-12 10:56:17,670
8,9,72,2014-02-20 11:43:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2437
9,10,32,2014-01-16 11:35:39,32.0,2014-01-16 11:35:49,561.0,2014-01-16 11:35:50,69.0,2014-01-16 11:36:17,32.0,...,2014-01-16 11:36:43,329.0,2014-01-16 11:36:56,69.0,2014-01-16 11:37:08,69.0,2014-01-16 11:37:09,329.0,2014-01-16 11:37:25,2238


**Пока в качестве признаков используем только посещенные сайты.**

**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

**Выделите сайты и переведите данные в текстовый формат, который можно подать на вход `CountVectorizer`.**

Пример: первая сессия состоит из сайтов 8, 11, 82, 68, 8, 8393, 8403, 932, 3260, 8. Надо это преобразовать в строку "8 11 82 68 8 8393 8403 932 3260 8".
Кстати, это можно сделать еще на этапе чтения данных из файла.



In [33]:
y = train_df['user_id']

In [34]:
sites = ['site' + str(i) for i in range(1,11)]

In [35]:
train_df_sites = train_df[sites].astype(str).apply(" ".join, axis=1)
test_df_sites = test_df[sites].astype(str).apply(" ".join, axis=1)

In [36]:
train_df_sites.replace(to_replace=['\.0', 'nan'], value='', regex=True, inplace=True)
test_df_sites.replace(to_replace=['\.0', 'nan'], value='', regex=True, inplace=True)

In [37]:
train_sessions_text = train_df_sites.map(lambda x: x.strip())
test_sessions_text = test_df_sites.map(lambda x: x.strip())

In [38]:
train_sessions_text[:10]

0                    8 11 82 68 8 8393 8403 932 3260 8
1              111 78 151 111 1401 151 1375 38 1401 97
2           11 3187 132 496 1969 504 3187 82 3191 3184
3            668 1965 598 1965 284 668 38 4451 4537 11
4    1943 1943 1943 1943 1943 1952 1952 1943 1943 1943
5          2941 2951 2958 2993 162 2967 85 7 2967 2967
6       9996 9996 307 9996 280 9996 9996 307 9996 9996
7          19823 19823 1510 32 19808 8 567 567 654 567
8                                                   72
9                    32 32 561 69 32 329 329 69 69 329
dtype: object

**Обучите `CountVectorizer` и получите 2 разреженные матрицы, соответствующие обучающей и тетсовой выборкам.**

In [39]:
count_vectorizer = CountVectorizer()

In [40]:
X_train_sparse = count_vectorizer.fit_transform(train_sessions_text)
X_test_sparse = count_vectorizer.transform(test_sessions_text)

In [41]:
X_train_sparse.shape, X_test_sparse.shape

((95319, 21523), (41177, 21523))

**<font color='red'> Вопрос 1:</font> Укажите число признаков (столбцов) в разреженных матрицах X_train_sparse и X_test_sparse.**

**Vowpal Wabbit любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 550). Поэтому придется применить `LabelEncoder`, да еще и +1 потом добавить (`LabelEncoder` переводит метки в диапазон от 0 до K-1).**

In [44]:
class_encoder = LabelEncoder()

In [56]:
y_for_vw = class_encoder.fit_transform(y) + 1

**Выделим обучающую и оставленную части исходной обучающей выборки.**

In [59]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_sparse, y_for_vw, test_size=0.3, 
                                                     random_state=7, stratify=y_for_vw)

**Реализуйте функцию, переводящую разреженную матрицу в формат Vowpal Wabbit.**

Вход:
 - X_sparse – разреженная матрица SciPy.sparse.csr_matrix
 - y (необяз.) - вектор ответов. Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
 - out_file – путь к файлу .vw, в который будет произведена запись
 
Детали:
- можно делать по-разному, но скорее всего поможет метод *nonzero* разреженной матрицы
- в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [ ]:
def sparse_matrix_to_vw(X_sparse, y=None, out_file='tmp.vw'):
    

**Примените написанную функцию к части обучащей выборки (X_train, y_train), к отложенной выборке (X_valid, y_valid), ко всей обучающей выборке (X_train_sparse, y_for_vw) и ко всей тестовой выборке X_test_sparse.**

In [ ]:
%%time
sparse_matrix_to_vw(X_train, y_train, '../../data/web_sessions_train_part.vw')
sparse_matrix_to_vw(X_valid, y_valid, '../../data/web_sessions_valid.vw')
sparse_matrix_to_vw(X_train_sparse, y_for_vw, '../../data/web_sessions_train.vw')
sparse_matrix_to_vw(X_test_sparse, out_file='../../data/web_sessions_test.vw')

In [ ]:
!head -2 ../../data/web_sessions_train_part.vw

In [ ]:
!head -2 ../../data/web_sessions_valid.vw

In [ ]:
!head -2 ../../data/web_sessions_train.vw

In [ ]:
!head -2 ../../data/web_sessions_test.vw

**Обучите модель на выборке kaggle_data/train_part.vw. Укажите, что решается задача классификации с 550 классами (--oaa), сделайте 3 прохода по выборке (--passes). Также укажите значение параметра b=26. Это число бит, используемых для хэширования, в данном случае нужно больше, чем 18 по умолчанию. Остальные параметры пока не меняйте, далее уже в свободном режиме соревнования можете попробовать другии функции потерь.**

In [ ]:
PATH_TO_DATA = '../../data'
train_part_vw = os.path.join(PATH_TO_DATA, 'web_sessions_train_part.vw')
valid_vw = os.path.join(PATH_TO_DATA, 'web_sessions_valid.vw')
train_vw = os.path.join(PATH_TO_DATA, 'web_sessions_train.vw')
test_vw = os.path.join(PATH_TO_DATA, 'web_sessions_test.vw')
model = os.path.join(PATH_TO_DATA, 'vw_model.vw')
pred = os.path.join(PATH_TO_DATA, 'vw_pred.csv')

In [ ]:
%%time
!vw ''' ВАШ КОД ЗДЕСЬ'''

**Запишите прогнозы на выборке *web_sessions_valid.vw* в *vw_pred.csv*.**

In [ ]:
%%time
!vw ''' ВАШ КОД ЗДЕСЬ'''

**Считайте прогнозы *vw_pred.csv*  из файла и посмотрите на долю правильных ответов на отложенной части.**

In [ ]:
vw_valid_pred = pd.read_csv ''' ВАШ КОД ЗДЕСЬ'''
accuracy_score ''' ВАШ КОД ЗДЕСЬ'''

**<font color='red'> Вопрос 2:</font> Какой получается доля правильных ответов на отложенной выборке?** 

**Обучите модель с теми же параметрами на всей обучающей выборке – *web_sessions_train.vw*.**

In [ ]:
%%time
!vw ''' ВАШ КОД ЗДЕСЬ'''

**Сделайте прогноз для тестовой выборки.**

In [ ]:
%%time
!vw ''' ВАШ КОД ЗДЕСЬ'''

**Запишите прогноз в файл, примените обратное преобразование меток (был `LabelEncoder` и потом +1 в меткам) и отправьте решение на Kaggle в [соревновании](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/data).**

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
vw_pred = pd.read_csv(pred, header=None)

In [ ]:
vw_submission = class_encoder.inverse_transform ''' ВАШ КОД ЗДЕСЬ'''

In [ ]:
write_to_submission_file(vw_subm, 'hw9_vw_submision.csv')